<a href="https://colab.research.google.com/github/dgromann/cl_intro/blob/main/exercises/HomeExercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home Exercise 3: Creating Neural Model in PyTorch

In this home exercise, you will first learn how to create a neural model in PyTorch and then you will train and improve a mini-implementation of an embedding model.


---

## **Exercise 3a: Neural Network Model**

First we need to import the neural network module of PyTorch:

In [5]:
import torch
import torch.nn as nn

We can use `nn.Linear(H_in, H_out)` to create a a linear layer. This will take a matrix of `(N, *, H_in)` dimensions and output a matrix of `(N, *, H_out)`. The `*` denotes that there could be arbitrary number of dimensions in between. The linear layer performs the operation `Ax+b`, where `A` and `b` are initialized randomly. If we don't want the linear layer to learn the bias parameters, we can initialize our layer with `bias=False`.

In [ ]:
# Create the inputs
input = torch.ones(2,3,4)
print("Input ", input)

# N* H_in -> N*H_out
linear = nn.Linear(4, 2)
linear_output = linear(input)
linear_output

In [ ]:
list(linear.parameters()) # Ax + b

Let's add an activation function:

In [ ]:
sigmoid = nn.Sigmoid()
output = sigmoid(linear_output)
output

Instead of creating intermediate layers and passing variables around, we can create a sequence:

In [ ]:
block = nn.Sequential(
    nn.Linear(4, 2),
    nn.Sigmoid()
)

input = torch.ones(2,3,4)
output = block(input)
output

---

## **Exercise 3b: Word Embeddings**


Instead of using predefined modules of nn we can define our own modules and build custom neural networks. As a toy example, we will convert words to word embeddings. The preprocessing below should be done more elegantly and not as simply as below.

In [ ]:
import string

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold."""

training_sentence = test_sentence.translate(str.maketrans('', '', string.punctuation)).lower().split()
print(training_sentence)

Next let's find our vocabulary, i.e., all the unique words in the training data:

In [ ]:
vocabulary = set(w for w in training_sentence)
vocabulary

We introduce a special token, `<unk>`, to tackle the words that are out of vocabulary. We could pick another string for our unknown token if we wanted. The only requirement here is that our token should be unique: we should only be using this token for unknown words. We will also add this special token to our vocabulary.

In [12]:
vocabulary.add("<unk>")

Now we will create the index for our vocabulary - one index to word and one word to index to make looking up words easier:

In [ ]:
ix_to_word = sorted(list(vocabulary))
word_to_ix = {word: ind for ind, word in enumerate(ix_to_word)}
word_to_ix

👋 ⚒ How can we now lookup which word is the fifth word in our index list?

In [ ]:
# Your code here


We will use a very simple solution of building trigrams to train our model.

In [106]:
trigrams = [([training_sentence[i], training_sentence[i + 1]], training_sentence[i + 2])
            for i in range(len(training_sentence)-2)]
print(trigrams)

[(['when', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege'), (['shall', 'besiege'], 'thy'), (['besiege', 'thy'], 'brow'), (['thy', 'brow'], 'and'), (['brow', 'and'], 'dig'), (['and', 'dig'], 'deep'), (['dig', 'deep'], 'trenches'), (['deep', 'trenches'], 'in'), (['trenches', 'in'], 'thy'), (['in', 'thy'], 'beautys'), (['thy', 'beautys'], 'field'), (['beautys', 'field'], 'thy'), (['field', 'thy'], 'youths'), (['thy', 'youths'], 'proud'), (['youths', 'proud'], 'livery'), (['proud', 'livery'], 'so'), (['livery', 'so'], 'gazed'), (['so', 'gazed'], 'on'), (['gazed', 'on'], 'now'), (['on', 'now'], 'will'), (['now', 'will'], 'be'), (['will', 'be'], 'a'), (['be', 'a'], 'totterd'), (['a', 'totterd'], 'weed'), (['totterd', 'weed'], 'of'), (['weed', 'of'], 'small'), (['of', 'small'], 'worth'), (['small', 'worth'], 'held'), (['worth', 'held'], 'then'), (['held', 'then'], 'being'), (['then', 'being'], 'asked'), (['being', 'asked'], 'where'), (['asked', 'where'

In [153]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, window_size, hidden_dim):
        super(NGramLanguageModeler, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.hidden_layer = nn.Sequential(
            nn.Linear(window_size * embedding_dim, hidden_dim),
            nn.ReLU()
        )
        self.output_layer = nn.Linear(hidden_dim, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        layer1 = self.hidden_layer(embeds)
        output = self.output_layer(layer1)
        probabilities = nn.functional.log_softmax(output, dim=1)
        return probabilities

👋 ⚒ Now let's train the model. Try to adapt the hyperparamters so that the total_loss is reduced. These include in the initial settings:

*   Dimensionality of the embeddings: 10
*   Dimensionality of the hidden layer: 128
*   Number of epochs: 10
*   Learning rate: 0.002
*   Loss function (`NLLLoss()` Negative Log Likelihood right now)

The window size can in this toy example not be changed, since we train on trigrams (so we only have two context words in the set).

The code cell before, the `class NGramLanguageModeler(nn.Module)` defines the exact setup of the network. For the setup one thing to change could be the  activation functions (ReLU and Log-Softmax right now) - others can be found [here](https://pytorch.org/docs/stable/nn.html) for sequential layers and here for other uses as [nn.functional](https://pytorch.org/docs/stable/nn.functional.html)).

Just for comparison, this is the initial output with the first settings of the notebook:

`See how loss decreases with each epoch:  [4.502952638980561, 4.453238930322428, 4.404419487556525, 4.356399970771992, 4.309086953644204, 4.262444785210938, 4.216389432417608, 4.170906921403598, 4.125928629816106, 4.081349758975274]
Loss of the last epoch: 4.081349758975274`




In [155]:
window_size = 2
embedding_dim = 10
hidden_dim = 128
num_epochs = 10

losses = []
# Negative log likelihood
loss_function = nn.NLLLoss()
ngram_model = NGramLanguageModeler(len(vocabulary), embedding_dim, window_size, hidden_dim)

# What do SGD and lr mean? What happenes if you change them?
optimizer = torch.optim.SGD(ngram_model.parameters(), lr=0.002)

for epoch in range(num_epochs):
    total_loss = 0
    for context, target in trigrams:
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        optimizer.zero_grad()

        # Step 3. Run the forward pass, getting probabilities over next
        # words - the size of this tensor is 87 corresponding to the size of the vocabulary
        probabilities = ngram_model.forward(context_idxs)

        # Step 4. Compute your loss function. The target word (gold standard label) needs to be
        # wrapped in a tensor.
        loss = loss_function(probabilities, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss / len(trigrams))

print("See how loss decreases with each epoch: ", losses)
print("Loss of the last epoch:", losses[-1])

See how loss decreases with each epoch:  [4.502952638980561, 4.453238930322428, 4.404419487556525, 4.356399970771992, 4.309086953644204, 4.262444785210938, 4.216389432417608, 4.170906921403598, 4.125928629816106, 4.081349758975274]
Loss of the last epoch: 4.081349758975274


👋 ⚒ Implement the cosine simmilarity for the trained embeddings. For this you need to choose too find the index of the two words you wish to compare, get their embeddings, convert them into numpy arrays, and run them through the cosine function provided.

In [ ]:
# Tensor to look up specific embeddings
lookup_tensor = torch.tensor(list(word_to_ix.values()), dtype=torch.long)

# Embedding for the first word in the vocabulary
lookup_embeds = ngram_model.embeddings(lookup_tensor)[0]
print(lookup_embeds)
#cosine = np.dot(A,B)/(norm(A, axis=1)*norm(B))